### Import Libraries that will be needed

In [1]:
import pandas as pd
import datetime

### Import data from .csv and reformat time

In [2]:
df_raw = pd.read_csv("Bethany_Raw.csv")

#Convert Time column to datetime.time format
df_raw.Time = pd.to_datetime(df_raw.Time, format='%H:%M:%S').dt.time 

### Trim to 24 hour range

In [3]:
def get_data_range(time_list):
    """
    Find the start and end indices for a 24 hour range starting between 10:00am and 10:05am
    Takes arguments time_list - a list of datetime.time objects
    Returns a tuple: (start_index, end_index)
    *Selects first instance of time between 10:00am and 10:05am encountered as start index*
    """
    lower_limit = datetime.time(10,0,0)
    upper_limit = datetime.time(10,5,0)
    start_index = 0
    
    for i in range(len(time_list)):
        if time_list[i] >= lower_limit and time_list[i] < upper_limit:
            start_index = i
            break
            
    return (start_index, start_index + 288) #288 is the number of rows until 24 hours later


In [4]:
# Trim raw data to 24 hour window
trim_start, trim_end = get_data_range(list(df_raw.Time))
df = df_raw[trim_start:trim_end]
df = df.reset_index(drop=True)
df_trimmed_copy = df.copy() # to be used in final output to excel 

### Create reference columns

In [5]:
# Create Boolean column indicating Night=True Day=False
# In most cases this should be just split into top/bottom half of df, 
# but implented as a function in case assumption is not met.
def is_night(x):
    """
    Take datetime.time object and return True if between 10pm and 10am, False otherwise 
    """
    night_start = datetime.time(22,0,0)
    night_end = datetime.time(10,0,0)
    if (x >= night_start and x < datetime.time(23, 59, 59)) or x < night_end:
        return True
    else:
        return False

In [6]:
df.isNight = df.Time.apply(is_night)

In [7]:
# Create boolean columns for fly sleeping state
def create_sleeping_columns(df):
    """
    Creates a new boolean column for each fly. True indicates fly is asleep for that window.
    Columns used for filters in data analysis. 
    """
    act_col = 11
    
    for i in range(5):
        df.loc[:,'fly' + str(i+1) + 'sleeping'] = df.iloc[:,i+act_col].apply(lambda x: x == 0)
    

In [8]:
create_sleeping_columns(df)

### Get day and night bouts and create dataframes

In [9]:
# Following code is used to generate two new dataframes: all_day_bouts_df and all_night_bouts_df
# These will be exported to two different sheets in excel. They will contain metabolic rate data 
# occuring during individual sleeping bouts, with one bout per column. 
# Different flies will be seperated with an empty column. 

def get_bout_indices(activity_list):
    """
    Takes a list, activity_list, and returns a list of tuples of the start/end indices in 
    which sleeping bouts occurr.
    I.e. if two sleeping bouts occured, the first from index 5 to 20, and the second from 
    index 30 to 40, this function will return [(5,20), (30,40)]
    """
    indices = []
    start_index = 1
    end_index = 1
    in_bout = False
    
    for i in range(len(activity_list)):
        if activity_list[i] == 0 and in_bout == False: 
            start_index = i
            in_bout = True
        if (activity_list[i] != 0 or i == len(activity_list)-1) and in_bout == True:
            end_index = i
            in_bout = False
            if i == len(activity_list)-1:
                indices.append((start_index, end_index+1))
            else:
                indices.append((start_index, end_index))
    return indices 
        

In [10]:
colnames = df.columns[11:16] # Ordered list of fly genotype name strings

def get_day_night_bouts(df, resp_colnum, activity_colnum):
    """
    Gets lists of mr rates for a single fly during during indepedent sleeping bouts 
    during either day or night. 
    Takes the dataframe, the colnumn number for resp data, column number for sleep data
    Returns a tuple of two lists of lists: resp data for daytime sleep bouts, 
    and resp data for nighttime sleep bouts
    """
    bout_indices = get_bout_indices(list(df.iloc[:,activity_colnum]))
    resp_bouts_day = []
    resp_bouts_night = []
    
    for i in bout_indices:
        if df.isNight[i[0]:i[1]].mean() >= 0.5:
            resp_bouts_night.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
        else:
            resp_bouts_day.append(list(df.iloc[:,resp_colnum])[i[0]:i[1]])
            
    if not resp_bouts_day:
        resp_bouts_day = ['none']
        
    if not resp_bouts_night:
        resp_bouts_night = ['none']
        
    return resp_bouts_day, resp_bouts_night

def get_all_bouts(df, colnames):
    """
    Returns a tuple of two dictionaries containing individual bouts for each fly 
    for 1) day sleep, 2) night sleep
    Dictionary format - key: genotype; value: list of lists containing mr data
    for individual sleeping bouts
    """
    mr_col = 4
    act_col = 11
    colnames = df.columns[11:16]
    all_day_bouts = {x: [] for x in colnames}
    all_night_bouts = {x: [] for x in colnames}
    
    for i in range(5):
        day_bouts, night_bouts = get_day_night_bouts(df, mr_col, act_col)
        all_day_bouts[df.columns[i+11]] += day_bouts 
        all_night_bouts[df.columns[i+11]] += night_bouts 
        mr_col+=1
        act_col+=1
        
    return all_day_bouts, all_night_bouts

In [11]:
day_bouts_dict, night_bouts_dict = get_all_bouts(df, colnames)

In [12]:
def get_all_bouts_df(day_bouts_dict, night_bouts_dict, colnames):
    """
    Takes dictionaries day_bouts_dict and night_bouts_dict and returns 
    a tuple of two dataframes containing the information in the dictionaries 
    properly formatted for excel output.
    """
    all_bouts_day_list = []
    all_bouts_night_list = []
    
    for i in colnames:
        # [[]] used to create empty lists between columns to indicate different flies
        all_bouts_day_list += day_bouts_dict[i] + [[]] 
        all_bouts_night_list += night_bouts_dict[i] + [[]]
        
    all_day_df = pd.DataFrame(all_bouts_day_list).transpose()
    all_night_df = pd.DataFrame(all_bouts_night_list).transpose()
    
    return all_day_df, all_night_df

In [13]:
# Create dataframes for day/night bouts
all_day_bouts_df, all_night_bouts_df = get_all_bouts_df(day_bouts_dict, night_bouts_dict, colnames)

In [14]:
all_day_bouts_df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1.039541,1.180896,1.284699,1.227334,1.185590,1.130278,0.986619,NaN,1.098401,NaN,...,1.366439,1.471703,1.284412,NaN,1.592927,1.589793,1.502461,1.616515,1.838532,NaN
1,1.427665,1.383591,1.285172,1.295008,1.095631,1.101771,1.009748,NaN,NaN,NaN,...,1.330130,1.332372,1.294859,NaN,1.610929,NaN,NaN,1.530170,1.890970,NaN
2,NaN,1.294287,NaN,1.206533,1.199461,1.044584,0.936784,NaN,NaN,NaN,...,1.410944,1.315407,1.380678,NaN,1.390133,NaN,NaN,1.537473,NaN,NaN
3,NaN,1.249192,NaN,NaN,NaN,1.026278,0.938204,NaN,NaN,NaN,...,1.311169,1.319408,1.336910,NaN,NaN,NaN,NaN,1.466006,NaN,NaN
4,NaN,1.326397,NaN,NaN,NaN,8.365838,0.951994,NaN,NaN,NaN,...,1.582302,1.280412,1.348364,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,1.331773,NaN,NaN,NaN,0.910308,0.950158,NaN,NaN,NaN,...,1.514232,1.296456,1.371576,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,1.148687,NaN,NaN,NaN,1.019505,0.945871,NaN,NaN,NaN,...,1.406155,1.219918,1.390585,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,1.122283,NaN,NaN,NaN,1.070171,0.945160,NaN,NaN,NaN,...,1.374179,1.174965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,1.248453,NaN,NaN,NaN,NaN,0.924083,NaN,NaN,NaN,...,1.469105,1.379992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,1.113087,NaN,NaN,NaN,NaN,1.021218,NaN,NaN,NaN,...,NaN,1.277233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get sleep profiles and create dataframe

In [15]:
# The following code is used to create the sleep_profile dataframe, 
# which will be exported as it's own sheet in the excel output. 
def get_single_sleep_profile(df, resp_colnum, activity_colnum):
    """
    Returns a list of tuples with sum of metabolic rate, the percentage sleep, 
    and total beam breaks for each hour in the 24 hour window, for one individual fly.
    """
    fly_activity = df.iloc[:,activity_colnum]
    fly_resp = df.iloc[:,resp_colnum]
    hourly_resp_sleep = []
    
    for i in range(24):
        activity = fly_activity[i*12:i*12+12] #12 rows are selected at a time to bin into 1-hour 
        resp = fly_resp[i*12:i*12+12]
        num_sleep_blocks = 0
        for i in activity:
            if i == 0:
                num_sleep_blocks += 1
        sleep_avg = num_sleep_blocks / 12 * 100
        hourly_resp_sleep.append((resp.sum(), sleep_avg, activity.sum()))
        
    return hourly_resp_sleep

def make_sleep_profile_colnames(df):
    """
    Create list of ordered column names for dataframe to be created from sleep_profile dictionary
    """
    colnames = ['Time']
    
    for i in range(11,16):
        colnames.append(df.columns[i] + ' MR Sum')
        colnames.append(df.columns[i] + ' Avg Sleep')
        colnames.append(df.columns[i] + ' Beam Breaks')
        
    return colnames

def make_sleep_profile_df(df):
    """
    Returns a dictionary containing a time index and the sleep profiles 
    (both metabolic rate sum and average sleep) for each fly in the dataframe.
    """
    mr_col = 4
    act_col = 11
    sleep_profile_dict = {}
    sleep_profile_dict['Time'] = [df.Time[x*12-1] for x in range(1, 25)]
    
    def add_to_dict(name, index):
        sleep_profile_dict[df.columns[act_col] + name] = [x[index] for x in sleep_profile]
        
    for i in range(5):
        sleep_profile = get_single_sleep_profile(df, mr_col, act_col)
        add_to_dict(' MR Sum', 0)
        add_to_dict(' Avg Sleep', 1)
        add_to_dict(' Beam Breaks', 2)
        mr_col+=1
        act_col+=1
        
    sleep_profile_all = pd.DataFrame(sleep_profile_dict, columns=make_sleep_profile_colnames(df))
    
    return sleep_profile_all



In [16]:
sleep_profile_df = make_sleep_profile_df(df)

In [17]:
sleep_profile_df

,Time,w1118 BL 6326 MR Sum,w1118 BL 6326 Avg Sleep,w1118 BL 6326 Beam Breaks,w1118 BL 6326.1 MR Sum,w1118 BL 6326.1 Avg Sleep,w1118 BL 6326.1 Beam Breaks,eaat2 e04636 MR Sum,eaat2 e04636 Avg Sleep,eaat2 e04636 Beam Breaks,eaat2 e04636.1 MR Sum,eaat2 e04636.1 Avg Sleep,eaat2 e04636.1 Beam Breaks,eaat2 e04636.2 MR Sum,eaat2 e04636.2 Avg Sleep,eaat2 e04636.2 Beam Breaks
0,10:56:04,13.666723,0.000000,936,18.943555,8.333333,431,16.985112,0.000000,521,18.626518,8.333333,89,20.879742,0.000000,438.0
1,11:56:04,14.268239,0.000000,1011,20.308204,0.000000,541,16.451909,0.000000,397,14.862725,83.333333,3,18.561291,0.000000,199.0
2,12:56:04,14.507468,0.000000,1128,20.116906,0.000000,507,16.343367,0.000000,310,16.466497,41.666667,50,18.079725,0.000000,144.0
3,13:56:04,15.105396,0.000000,1111,19.625666,0.000000,417,16.287813,0.000000,248,17.933557,16.666667,152,18.222576,0.000000,122.0
4,14:56:04,14.410743,0.000000,864,19.774922,0.000000,447,16.533504,0.000000,235,14.094162,100.000000,0,17.426245,0.000000,109.0
5,15:56:04,15.773512,75.000000,68,18.800395,0.000000,423,15.486906,0.000000,266,15.227216,41.666667,56,17.478851,0.000000,119.0
6,16:56:04,14.073482,100.000000,0,18.979456,0.000000,361,14.596119,0.000000,259,16.543512,16.666667,74,18.212323,25.000000,77.0
7,17:56:04,15.142213,66.666667,18,19.021843,0.000000,245,17.382679,8.333333,200,18.153170,58.333333,25,19.557847,16.666667,64.0
8,18:56:04,15.210787,41.666667,41,19.139298,0.000000,310,17.182623,0.000000,206,15.557176,100.000000,0,20.330804,33.333333,114.0
9,19:56:05,20.560867,83.333333,12,20.722830,0.000000,494,18.735982,0.000000,247,13.717040,100.000000,0,18.886279,0.000000,80.0


### Create summary dataframes

In [18]:
# The following code is used to analyze the data processed above, and generate three dataframes, 
# which will become three sheets in the final excel output. 

#Total Sleep 
def get_sleep_minutes_df(df, colnames):
    """
    Create summary df of minutes of sleep for each fly. 
    Exported as sheet to excel.
    """
    total_sleep_list = [[x, ] for x in colnames]
    sleep_col = 16
    column_names = ['Fly', 
                    'Total Sleep Min',
                    'Total Day Sleep Min', 
                    'Total Night Sleep Min']
    
    def add_to_list(data):
        total_sleep_list[i].append(data)
    
    for i in range(len(colnames)):
        sleep_iloc = df.iloc[:,i+sleep_col]
        add_to_list(sleep_iloc.sum()*5) # data are in 5min bins, multiply by 5 to make per minute
        add_to_list(sleep_iloc[df.isNight == False].sum()*5)
        add_to_list(sleep_iloc[df.isNight == True].sum()*5)
        
    return pd.DataFrame(total_sleep_list, columns=column_names)

In [19]:
sleep_minutes_df = get_sleep_minutes_df(df, colnames)

In [20]:
sleep_minutes_df

,Fly,Total Sleep Min,Total Day Sleep Min,Total Night Sleep Min
0,w1118 BL 6326,820,290,530
1,w1118 BL 6326.1,645,5,640
2,eaat2 e04636,645,5,640
3,eaat2 e04636.1,995,380,615
4,eaat2 e04636.2,515,55,460


In [21]:
# Wake Sleep MR
def get_wake_sleep_mr_df(df, colnames):
    """
    Create summary df of average metabolic rates during selected time periods for each fly. 
    Exported as sheet to excel. 
    """
    wake_sleep_mr_list = [[x, ] for x in colnames]
    mr_col = 4
    sleep_col = 16
    column_names = ['Fly',
                    'Mean Wake MR',
                    'Mean Sleep MR',
                    'Mean Day MR', 
                    'Mean Night MR', 
                    'Mean Day Sleep MR',
                    'Mean Night Sleep MR']
    
    def add_to_list(data):
        wake_sleep_mr_list[i].append(data)
    
    for i in range(len(colnames)):
        mr_iloc = df.iloc[:,i+mr_col]
        sleep_iloc = df.iloc[:,i+sleep_col]
        
        add_to_list(mr_iloc[sleep_iloc == False].mean())
        add_to_list(mr_iloc[sleep_iloc == True].mean())
        add_to_list(mr_iloc[df.isNight == False].mean())
        add_to_list(mr_iloc[df.isNight == True].mean())
        add_to_list(mr_iloc[sleep_iloc == True][df.isNight == False].mean())
        add_to_list(mr_iloc[sleep_iloc == True][df.isNight == True].mean())            
        
    return pd.DataFrame(wake_sleep_mr_list, columns=column_names)

In [22]:
wake_sleep_mr_df = get_wake_sleep_mr_df(df, colnames)

In [23]:
wake_sleep_mr_df

,Fly,Mean Wake MR,Mean Sleep MR,Mean Day MR,Mean Night MR,Mean Day Sleep MR,Mean Night Sleep MR
0,w1118 BL 6326,1.201862,1.170727,1.235784,1.132480,1.265810,1.118700
1,w1118 BL 6326.1,1.631928,1.023106,1.646857,1.071596,1.098401,1.022518
2,eaat2 e04636,1.380552,0.890217,1.413147,0.908699,1.461901,0.885750
3,eaat2 e04636.1,1.453698,1.159332,1.360580,1.140018,1.242371,1.108024
4,eaat2 e04636.2,1.683387,1.538550,1.607447,1.655728,1.596901,1.531573


In [24]:
def get_mean_hourly_sleep_mr_df(sleep_profile, colnames):
    """
    Creates summary df for mean hourly sleeping metabolic rate for each fly. 
    Exported as sheet to excel. 
    """
    mr_hourly = sleep_profile.iloc[:,1::3]
    # sleep profile is split into top half: day; bottom half: night
    night_bool_list = [x > len(mr_hourly)/2-1 for x in range(len(mr_hourly))]
    mr_hourly.isNight = pd.Series(night_bool_list)      
    mean_hourly_list = [[x, ] for x in colnames]
    column_names = ['Fly', 
                    'Mean Hourly MR Total', 
                    'Mean Hourly MR Day', 
                    'Mean Hourly MR Night']
    
    def add_to_list(data):
        mean_hourly_list[i].append(data)
    
    for i in range(len(colnames)):
        mr_iloc = mr_hourly.iloc[:,i]
        add_to_list(mr_iloc.mean())
        add_to_list(mr_iloc[mr_hourly.isNight == False].mean())
        add_to_list(mr_iloc[mr_hourly.isNight == True].mean())
        
    return pd.DataFrame(mean_hourly_list, columns=column_names)

In [25]:
mean_hourly_sleep_mr_df = get_mean_hourly_sleep_mr_df(sleep_profile_df, colnames)

In [26]:
mean_hourly_sleep_mr_df

,Fly,Mean Hourly MR Total,Mean Hourly MR Day,Mean Hourly MR Night
0,w1118 BL 6326,14.209586,14.829413,13.589760
1,w1118 BL 6326.1,16.310717,19.762282,12.859151
2,eaat2 e04636,13.931072,16.957759,10.904384
3,eaat2 e04636.1,15.003590,16.326964,13.680216
4,eaat2 e04636.2,19.579051,19.289370,19.868732


## Export to .xlsx

In [27]:
# Convert datetime.time columns back to str for correct formatting in Excel
sleep_profile_df.Time = sleep_profile_df.Time.astype(str)
df_trimmed_copy.Time = df_trimmed_copy.Time.astype(str)

def excel_out(df, name, **kwargs):
    """
    Wrapper function - Takes a dataframe, and a desired sheet name (str).
    Sends to new sheet in excel output
    """
    df.to_excel(writer, sheet_name = name, index = False, **kwargs)
    
writer = pd.ExcelWriter('test_out.xlsx', engine='xlsxwriter')
excel_out(df_trimmed_copy, 'Trimmed Analysis')
excel_out(all_day_bouts_df, 'All Day Bouts', header = False)
excel_out(all_night_bouts_df, 'All Night Bouts', header = False)
excel_out(sleep_profile_df, 'Sleep Profile')
excel_out(sleep_minutes_df, 'Min. of Sleep')
excel_out(wake_sleep_mr_df, 'Wake Sleep MR')
excel_out(mean_hourly_sleep_mr_df, 'Mean Hourly Sleep')
writer.save()